In [1]:
from google.colab import drive
drive.mount('/content/drive')

import sys, os
sys.path.append('/content/drive/Othercomputers/My_Mac/sentinel')
os.chdir('/content/drive/Othercomputers/My_Mac/sentinel')

!pip install -q torch torchvision


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import torch
import torch.nn as nn
import torch.jit
import numpy as np
import time
from pathlib import Path

from src.python.models.pointnet2 import PointNet2SemanticSegmentation
from src.python.config.training_config import TrainingConfig


In [11]:
class DeploymentModel(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base_model = base_model
        self.input_channels = 4  # Always expect 4 (XYZI)

    def forward(self, points):
        B, N, C = points.shape
        if C == 3:
            intensity = torch.ones(B, N, 1, device=points.device)
            points = torch.cat([points, intensity], dim=-1)
        elif C != 4:
            raise ValueError(f"Expected 3 or 4 input channels, got {C}")
        return self.base_model(points)


In [12]:
config = TrainingConfig()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cpu


In [13]:
print("Loading trained model...")
base_model = PointNet2SemanticSegmentation(num_classes=config.get('model.num_classes'))

checkpoint_path = os.path.join(config.get('paths.checkpoint_dir'), 'best_model.pth')
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    base_model.load_state_dict(checkpoint['model_state_dict'])
    print(f"Loaded model from epoch {checkpoint['epoch']}")
else:
    print("⚠️ No trained model found – using random weights")

model = DeploymentModel(base_model).to(device).eval()
print("Model wrapped for deployment")


Loading trained model...
Loaded model from epoch 1
Model wrapped for deployment


In [14]:
print("\nAttempting to script the model (may show warnings)...")

try:
    scripted_model = torch.jit.script(model)
    print("✓ Model successfully scripted!")
    use_scripting = True
except Exception as e:
    print(f"Scripting failed: {e}")
    print("Falling back to tracing...")
    use_scripting = False



Attempting to script the model (may show warnings)...
Scripting failed: Can't redefine method: forward on class: __torch__.src.python.models.pointnet2.___torch_mangle_140.PointNetSetAbstraction (of Python compilation unit at: 0x35bd3440)
Falling back to tracing...


In [15]:
if not use_scripting:
    print("\nConverting model to TorchScript using tracing...")
    example_input = torch.randn(1, 50000, 4).to(device)

    with torch.no_grad():
        _ = model(example_input)
        traced_model = torch.jit.trace(model, example_input, check_trace=False)

    scripted_model = traced_model
    print("Model traced successfully!")
else:
    print("Using scripted model")



Converting model to TorchScript using tracing...


/tmp/ipython-input-129345524.py:9: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if C == 3:
/tmp/ipython-input-129345524.py:12: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  elif C != 4:


Model traced successfully!


In [16]:
print("\nVerifying exported model...")
test_sizes = [10000, 25000, 50000]

for size in test_sizes:
    test_input = torch.randn(1, size, 4).to(device)
    try:
        with torch.no_grad():
            original_output = model(test_input)
            exported_output = scripted_model(test_input)

        assert original_output.shape == exported_output.shape, "Shape mismatch"
        diff = torch.abs(original_output - exported_output)
        print(f"Points: {size:,}")
        print(f"  Max diff: {diff.max():.6f} | Mean diff: {diff.mean():.6f}")
    except Exception as e:
        print(f"  Error: {e}")

print("\n✓ Model export completed (check above for any warnings)")



Verifying exported model...
Points: 10,000
  Max diff: 47.125271 | Mean diff: 2.869396
Points: 25,000
  Max diff: 37.051289 | Mean diff: 3.129232
Points: 50,000
  Max diff: 43.401329 | Mean diff: 3.529582

✓ Model export completed (check above for any warnings)


In [17]:
class SimplifiedDeploymentModel(nn.Module):
    def __init__(self, checkpoint_path, num_classes=20):
        super().__init__()
        from src.python.models.pointnet2 import PointNet2SemanticSegmentation
        self.model = PointNet2SemanticSegmentation(num_classes)
        if os.path.exists(checkpoint_path):
            checkpoint = torch.load(checkpoint_path, map_location='cpu')
            self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.eval()

    @torch.jit.export
    def forward(self, points: torch.Tensor) -> torch.Tensor:
        with torch.no_grad():
            return self.model(points)

if use_scripting == False:
    print("\nTrying simplified model...")
    simplified_model = SimplifiedDeploymentModel(checkpoint_path).to(device).eval()
    test_input = torch.randn(1, 1000, 4).to(device)
    with torch.no_grad():
        test_output = simplified_model(test_input)
    print(f"Simplified model output shape: {test_output.shape}")



Trying simplified model...
Simplified model output shape: torch.Size([1, 1000, 20])


In [20]:
print("\nOptimizing model for deployment...")
if hasattr(torch.jit, 'optimize_for_inference'):
    optimized_model = torch.jit.optimize_for_inference(scripted_model)
else:
    print("optimize_for_inference not available")
    optimized_model = scripted_model
optimized_model = optimized_model.cpu().eval()
print("Model optimized for inference")



Optimizing model for deployment...
Model optimized for inference


In [21]:
print("\nBenchmarking model performance...")

def benchmark_model(model, input_size, num_runs=5, device='cpu'):
    model = model.to(device).eval()
    times = []
    for _ in range(num_runs):
        x = torch.randn(1, input_size, 4).to(device)
        if device == 'cuda': torch.cuda.synchronize()
        start = time.time()
        with torch.no_grad(): _ = model(x)
        if device == 'cuda': torch.cuda.synchronize()
        times.append((time.time() - start) * 1000)
    return {
        'mean': np.mean(times), 'std': np.std(times),
        'min': np.min(times), 'max': np.max(times)
    }

configs = [(10000, 'cpu'), (25000, 'cpu'), (50000, 'cpu')]

print("\nPerformance Benchmarks (CPU):")
print("="*60)
print(f"{'Points':<10}{'Mean (ms)':<12}{'Std':<12}{'Min':<12}{'Max':<12}")
print("-"*60)
for num_pts, dev in configs:
    stats = benchmark_model(optimized_model, num_pts, device=dev)
    print(f"{num_pts:<10}{stats['mean']:<12.2f}{stats['std']:<12.2f}{stats['min']:<12.2f}{stats['max']:<12.2f}")
print("="*60)



Benchmarking model performance...

Performance Benchmarks (CPU):
Points    Mean (ms)   Std         Min         Max         
------------------------------------------------------------
10000     2588.32     1907.80     1206.90     6139.68     
25000     3279.00     178.21      3007.57     3564.62     
50000     6526.70     423.48      6109.66     7052.01     


In [22]:
output_dir = Path(config.get('paths.checkpoint_dir'))
output_dir.mkdir(parents=True, exist_ok=True)
output_path = output_dir / 'sentinel_model.pt'

print(f"\nSaving optimized model to: {output_path}")
try:
    optimized_model.save(str(output_path))
    print("✓ Model saved successfully")
except:
    torch.jit.save(optimized_model, str(output_path))
    print("✓ Model saved using fallback method")

if output_path.exists():
    file_size = output_path.stat().st_size / (1024 * 1024)
    print(f"Model file size: {file_size:.2f} MB")
else:
    print("⚠️ Warning: Model file not saved")



Saving optimized model to: /content/drive/MyDrive/project-sentinel/models/sentinel_model.pt
✓ Model saved successfully
Model file size: 2.70 MB


In [23]:
print("\nTesting model loading...")

try:
    loaded_model = torch.jit.load(str(output_path)).eval()
    test_input = torch.randn(1, 5000, 4)
    with torch.no_grad():
        test_output = loaded_model(test_input)
    print(f"Test output shape: {test_output.shape}")
    assert test_output.shape == (1, 5000, config.get('model.num_classes'))
    print("✓ Model successfully loaded and tested!")
except Exception as e:
    print(f"⚠️ Error: {e}")



Testing model loading...
⚠️ Error: required keyword attribute 'value' is undefined


In [24]:
print("\n" + "="*60)
print("Optional: Export to ONNX format")
print("="*60)

try:
    import torch.onnx
    onnx_path = output_dir / 'sentinel_model.onnx'
    dummy_input = torch.randn(1, 10000, 4).to(device)
    model.eval()
    torch.onnx.export(
        model, dummy_input, str(onnx_path),
        export_params=True, opset_version=11, do_constant_folding=True,
        input_names=['points'], output_names=['predictions'],
        dynamic_axes={'points': {0: 'batch', 1: 'points'}, 'predictions': {0: 'batch', 1: 'points'}}
    )
    print(f"✓ ONNX exported: {onnx_path} ({onnx_path.stat().st_size/1024/1024:.2f} MB)")
except Exception as e:
    print(f"ONNX export failed: {e}")



Optional: Export to ONNX format


/tmp/ipython-input-728438755.py:10: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(
/tmp/ipython-input-129345524.py:9: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if C == 3:
/tmp/ipython-input-129345524.py:12: TracerWarning: 

ONNX export failed: ONNX symbolic expected a constant value of the 'high' argument, got '184 defined in (%184 : Long(device=cpu) = onnx::Gather[axis=0](%181, %183), scope: __main__.DeploymentModel::/src.python.models.pointnet2.PointNet2SemanticSegmentation::base_model/src.python.models.pointnet2.PointNetSetAbstraction::sa1 # /content/drive/Othercomputers/My_Mac/sentinel/src/python/models/pointnet2_utils.py:56:0
)'  [Caused by the value '184 defined in (%184 : Long(device=cpu) = onnx::Gather[axis=0](%181, %183), scope: __main__.DeploymentModel::/src.python.models.pointnet2.PointNet2SemanticSegmentation::base_model/src.python.models.pointnet2.PointNetSetAbstraction::sa1 # /content/drive/Othercomputers/My_Mac/sentinel/src/python/models/pointnet2_utils.py:56:0
)' (type 'Tensor') in the TorchScript graph. The containing node has kind 'onnx::Gather'.] 
    (node defined in /content/drive/Othercomputers/My_Mac/sentinel/src/python/models/pointnet2_utils.py(56): farthest_point_sample
/content/d

In [25]:
summary = f"""
========================================
DEPLOYMENT PREPARATION SUMMARY
========================================

Model: PointNet++ with Feature Propagation
Num Classes: {config.get('model.num_classes')}
Input Format: [B, N, 4]
Output Format: [B, N, {config.get('model.num_classes')}]

Exported TorchScript Model: {output_path}
Size: {file_size:.2f} MB
Export Method: {'Script' if use_scripting else 'Trace'}

Benchmarks: See cell 12

Next Steps:
- Use in C++ with LibTorch
- Path: models/sentinel_model.pt
- Include header: model_info.h
========================================
"""

print(summary)

with open(output_dir / 'deployment_summary.txt', 'w') as f:
    f.write(summary)

print("✓ Deployment summary saved!")



DEPLOYMENT PREPARATION SUMMARY

Model: PointNet++ with Feature Propagation
Num Classes: 20
Input Format: [B, N, 4]
Output Format: [B, N, 20]

Exported TorchScript Model: /content/drive/MyDrive/project-sentinel/models/sentinel_model.pt
Size: 2.70 MB
Export Method: Trace

Benchmarks: See cell 12

Next Steps:
- Use in C++ with LibTorch
- Path: models/sentinel_model.pt
- Include header: model_info.h

✓ Deployment summary saved!
